# Preprocessing
1. Convert text to lowercase 
2. Remove URLs, mentions, and special characters (besides hashtags and emojis)
3. Remove stop words
4. Perform stemming/lemmatization

Download the spaCy model
Can be installed via `python -m spacy download en_core_web_sm`

Download Pre-Trained-Language Model: `wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin`

In [114]:
import pandas as pd
import re
import spacy
import fasttext
from googletrans import Translator

In [115]:
# Load the JSON file
df_posts = pd.read_json('../data/dataset.json')
df_posts.head()

,timestamp,text,text_id,user,user_id
0,2024-10-31,Running a business means juggling countless ad...,2018569761,danielwoodard,1077866112
1,2024-10-31,Liz Truss is walking in the lingering shadow o...,2092717718,nelsonjacqueline,1089670430
2,2024-10-31,The UK is bracing for war as government buildi...,2059143248,ihooper,1007478642
3,2024-10-31,Marrying a second or third cousin once removed...,2008209828,wrightnicholas,1039258480
4,2024-10-31,It's truly disgraceful how the Indian National...,2001239278,michael51,1021455936


In [116]:
# Info this can take over 30m!!
# spacy.prefer_gpu()  # Prefers GPU but doesn't crash if unavailable
# nlp = spacy.load("en_core_web_trf")
# def extract_entities(text):
#     """
#     Extracts named entities from text using SpaCy's NER model.

#     Args:
#     text (str): The text from which to extract named entities.

#     Returns:
#     list: A list of tuples where each tuple contains (entity_text, entity_label).
#     """
#     if not text or pd.isna(text):
#         return []  # Return an empty list if text is missing

#     # Process text with SpaCy
#     doc = nlp(text)

#     # Extract entity text and labels
#     entities = [(ent.text, ent.label_) for ent in doc.ents]

#     return entities

# def preprocess_text(text):
#     """
#     Preprocesses text by removing URLs and emojis while keeping mentions and hashtags intact.

#     Args:
#     text (str): The original text.

#     Returns:
#     str: Preprocessed text.
#     """
#     if not text or pd.isna(text):
#         return ""  # Return empty string if text is missing

#     # Remove URLs
#     text = re.sub(r"http\S+|www\S+|https\S+", '', text)
#     # Remove emojis
#     text = emoji.replace_emoji(text, replace="")
    
#     return text.strip()

# # Create a preprocessed text column
# df_posts['preprocessed_text'] = df_posts['text'].apply(preprocess_text)

# # Apply NER extraction on the preprocessed text
# df_posts['entities'] = df_posts['preprocessed_text'].apply(extract_entities)

Check for rows with no text

In [117]:
# Display rows where 'text' is missing (NaN)
missing_text_rows = df_posts[df_posts['text'].isnull()]
missing_text_rows

,timestamp,text,text_id,user,user_id


### Convert variables

Move hashtags to a new column 'hashtags'

In [118]:
# Apply re.findall() to each row in the 'text' column to extract hashtags
df_posts['hashtags'] = df_posts['text'].apply(lambda x: re.findall(r'#\w+', x) if isinstance(x, str) else [])

Move mentions to a new column 'mentions'

In [119]:
# Extract mentions from the 'text' column, remove the '@' symbol, and create a new column 'mentions'
df_posts['mentions'] = df_posts['text'].apply(lambda x: [mention[1:] for mention in re.findall(r'@\w+', x)] if isinstance(x, str) else [])

Convert text to lowercase

In [120]:
df_posts['text'] = df_posts['text'].str.lower()

Remove Date from Timestamp

In [121]:
# All Dates are the same
unique_dates = df_posts['timestamp'].dt.date.unique()
unique_dates

array([datetime.date(2024, 10, 31)], dtype=object)

In [122]:
df_posts['timestamp'] = df_posts['timestamp'].dt.time
df_posts.head()

,timestamp,text,text_id,user,user_id,hashtags,mentions
0,00:00:00,running a business means juggling countless ad...,2018569761,danielwoodard,1077866112,"[#HRtech, #businessmanagement]",[]
1,00:00:00,liz truss is walking in the lingering shadow o...,2092717718,nelsonjacqueline,1089670430,[#politics],[]
2,00:00:00,the uk is bracing for war as government buildi...,2059143248,ihooper,1007478642,"[#Ukrainewashed, #WarPreparedness]",[]
3,00:00:00,marrying a second or third cousin once removed...,2008209828,wrightnicholas,1039258480,"[#FamilyTree, #GeneticFacts]",[]
4,00:00:00,it's truly disgraceful how the indian national...,2001239278,michael51,1021455936,[#RationChorCongress],[]


### Remove URLs, Mentions, and Special Characters

Leave apostrophes in here for better lemmatization performance

In [123]:
# Pre-compile regex patterns
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\U00002700-\U000027BF"  # dingbats
    u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
    u"\U00002600-\U000026FF"  # miscellaneous symbols
    u"\U00002B50-\U00002B55"  # stars
    "]+", flags=re.UNICODE)

url_pattern = re.compile(r'http\S+|www\S+|https\S+')  # Removes URLs
mention_pattern = re.compile(r'@\w+')  # Removes mentions
punctuation_pattern = re.compile(r"[^\w\s'’]")  # Removes punctuation but keeps apostrophes
number_pattern = re.compile(r'\d+')  # Removes numbers
whitespace_pattern = re.compile(r'\s+')  # Removes excessive whitespace
hashtag_pattern = re.compile(r'#\w+')  # Removes hashtags and all text after them

# Removes spaces between letters in a single word
letter_spacing_pattern = re.compile(r'(\b\w)(?:\s+)(\w\b)')

# List of terms to remove
remove_tw_terms = ["cc", "cx", "ct", "dm", "ht", "mt", "prt", "rt", "followback", "follow back", "fb", "retweet", "retweets"]

# Compile regex to match terms (case insensitive and whole word)
remove_terms_pattern = re.compile(r'\b(' + '|'.join(remove_tw_terms) + r')\b')

# Updated regex for matching spaced-out letters (e.g., "s h a r e")
letter_spacing_pattern = re.compile(r'(\b(?:\w\s)+\w\b)')

# Function to merge spaced-out letters
def merge_spaced_letters(match):
    # Remove spaces within the matched group
    return match.group(0).replace(' ', '')

def normalize_full_width(text):
    # Convert full-width characters to half-width
    return ''.join(
        chr(ord(char) - 0xFEE0) if 0xFF01 <= ord(char) <= 0xFF5E else char
        for char in text
    )

# Updated preprocessing function
def preprocess_text(text):
    if pd.isna(text):
        return "", []  # Handle missing values gracefully
    
    # Normalize full-width characters
    text = normalize_full_width(text)

    # Extract emojis
    emojis = emoji_pattern.findall(text)  # List of emojis

    # Remove hashtags and text following them
    text = hashtag_pattern.sub('', text)

    # Remove emojis, URLs, mentions, punctuation, and numbers
    text = emoji_pattern.sub('', text)  # Remove emojis
    text = url_pattern.sub('', text)  # Remove URLs
    text = mention_pattern.sub('', text)  # Remove mentions
    text = punctuation_pattern.sub('', text)  # Remove punctuation
    text = number_pattern.sub('', text)  # Remove numbers

    # Remove specific terms (CC, CX, CT, DM, etc.)
    text = remove_terms_pattern.sub('', text)

    # Normalize letter spacing (e.g., "s h a r e" -> "share")
    text = letter_spacing_pattern.sub(merge_spaced_letters, text)

    # Remove excessive whitespace and trim
    text = whitespace_pattern.sub(' ', text).strip()

    return text, emojis

# Apply preprocessing to create new columns
df_posts[['text', 'emojis']] = df_posts['text'].apply(lambda x: pd.Series(preprocess_text(x)))

# Display head to check the results
df_posts[['text', 'emojis']].head()

,text,emojis
0,running a business means juggling countless ad...,[]
1,liz truss is walking in the lingering shadow o...,[]
2,the uk is bracing for war as government buildi...,[🇺🇦]
3,marrying a second or third cousin once removed...,[🧬]
4,it's truly disgraceful how the indian national...,"[🤦, ♂]"


Test output

In [124]:
df_posts.to_csv('../output/testing.csv', index=False)

## Check for duplicates

In [125]:
# Total duplicate rows
total_duplicate_rows = df_posts['text'].duplicated(keep=False).sum()

# Number of unique duplicate tweets
unique_duplicate_tweets = (df_posts['text'].value_counts() > 1).sum()

print(f"Total duplicate rows: {total_duplicate_rows}")
print(f"Unique duplicate tweets: {unique_duplicate_tweets}")

Total duplicate rows: 40239
Unique duplicate tweets: 16405


# Duplicate stats

In [126]:
# Get the frequency distribution of tweets
frequency_distribution = df_posts['text'].value_counts()

# Filter for only duplicates (frequency > 1)
duplicate_tweet_frequencies = frequency_distribution[frequency_distribution > 1]

# Summary statistics
print(duplicate_tweet_frequencies.describe())

# How many tweets are repeated more than 5 times?
highly_duplicated = (duplicate_tweet_frequencies > 5).sum()
print(f"Number of tweets repeated more than 5 times: {highly_duplicated}")

count    16405.000000
mean         2.452850
std          2.188017
min          2.000000
25%          2.000000
50%          2.000000
75%          3.000000
max        130.000000
Name: count, dtype: float64
Number of tweets repeated more than 5 times: 150


Remove duplicate tweets only if the same person posted the same tweet (spam) but extract frequency

In [127]:
# Step 1: Group by 'user' and 'text' and calculate the frequency of each combination
df_posts['frequency'] = df_posts.groupby(['user', 'text'])['text'].transform('count')

# Step 2: Drop duplicates based on 'user' and 'text' (keeping the first occurrence)
df_posts = df_posts.drop_duplicates(subset=['user', 'text'], keep='first').reset_index(drop=True)

In [128]:
df_posts[df_posts['user'] == 'reginabarnes']

,timestamp,text,text_id,user,user_id,hashtags,mentions,emojis,frequency
48,00:00:19,wwf no mercy usa variant nintendo n cart only ...,2053226516,reginabarnes,1068051022,[],[],[],86


Indentify language

In [129]:
# Load the pre-trained language identification model
model = fasttext.load_model('../models/lid.176.bin')

# Function to detect language using FastText
def detect_language_fasttext(text, threshold=0.8):
    # Skip tweets with fewer than 3 words
    if len(text.split()) < 3:
        return None  # Skip these tweets entirely
    try:
        predictions = model.predict(text)  # Predict the language
        lang_code = predictions[0][0].replace("__label__", "")  # Extract language code
        return lang_code
    except Exception as e:
        return None  # Skip on exception

# Apply language detection to the 'text' column
df_posts['language'] = df_posts['text'].apply(detect_language_fasttext)

# Filter non-English tweets (ignoring None values)
non_english_tweets = df_posts[(df_posts['language'].notna()) & (df_posts['language'] != 'en')]

# Display the number of non-English tweets and a sample
print(f"Number of non-English tweets: {len(non_english_tweets)}")
print(non_english_tweets[['text', 'language']])

# Save the non-English tweets to a CSV file
non_english_tweets.to_csv('../output/non_english_tweets.csv', index=False)

Number of non-English tweets: 865
                                                    text language
48     wwf no mercy usa variant nintendo n cart only ...       hu
187    votem sempre na tag se deixem decidir o futuro...       pt
256    aunque ya no puedo votar en otras plataformas ...       es
330    vote no site e na tag escutem no app e deem vi...       pt
529    when life gives you lemons throw them back and...       th
...                                                  ...      ...
68273  mil eo jungkook vai cantar no seu aniversário ...       pt
68365                              la bellezza è ovunque       it
68387  se esse tweet chegar a o jungkook vai fazer um...       pt
68451  a votação tá on votem na hashtag e escutem as ...       pt
68514  o que é isso a katy perry se aposentou acredit...       pt

[865 rows x 2 columns]


Count the number of appearances of each Language

In [134]:
language_counts = non_english_tweets['language'].value_counts()

print("Number of tweets per non-english language:")
print(language_counts)

Number of tweets per non-english language:
language
pt     546
es     143
th      74
id      17
sw      11
de      10
tl       9
it       5
ko       4
ta       4
fr       3
ar       3
fi       3
pl       3
ms       3
tr       3
ja       2
ru       2
nap      2
bn       2
lt       2
ca       2
no       2
hi       2
nl       2
mr       1
uz       1
hu       1
sv       1
ml       1
la       1
Name: count, dtype: int64


Translate the tweets

In [ ]:
# Initialize translator
translator = Translator()

languages_to_translate = ['pt', 'es', 'th', 'id']  # List of language codes
filtered_tweets = non_english_tweets[non_english_tweets['language'].isin(languages_to_translate)]

# Function to translate text
def translate_text(text):
    try:
        return translator.translate(text, dest='en').text
    except Exception as e:
        return text  # Return original text if translation fails

# Translate the tweets in the filtered DataFrame
filtered_tweets['translated_text'] = filtered_tweets['text'].apply(translate_text)

# Save the translated tweets to a CSV file
output_path = '../output/translated_tweets.csv'
filtered_tweets.to_csv(output_path, index=False)

# Display the number of translated tweets
print(f"Number of tweets translated: {len(filtered_tweets)}")
print(f"Translated tweets saved to: {output_path}")

Number of tweets translated: 780
Translated tweets saved to: ../output/translated_tweets.csv


/var/folders/m6/2lgmm8516r37yq5gnyyd84g00000gn/T/ipykernel_25383/754318150.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tweets['translated_text'] = filtered_tweets['text'].apply(translate_text)


## Use spacy

Load en_core_web_sm for spacy

In [130]:
nlp = spacy.load("en_core_web_sm")

### Lemmatization

Use lemmatization since stemming can lead to less accurate results (even non-words)

In [131]:
# Info this takes about 2.5m!!
df_posts['text'] = df_posts['text'].apply(
        lambda text: ' '.join([token.lemma_ for token in nlp(text)])
)

KeyboardInterrupt: 

### Create output for sentiment analysis

Keep stopwords

In [19]:
df_posts.to_csv('../output/preprocessed_for_SA.csv', index=False)

### Remove stopwords

In [20]:
df_posts['text'] = df_posts['text'].fillna("").apply(
    lambda text: ' '.join([token.text for token in nlp.make_doc(text) if not token.is_stop])
)

### Remove Apostrophes

In [21]:
df_posts['text'] = df_posts['text'].str.replace(r"[’']", "", regex=True)

### Check again for empty rows

In [22]:
# Display rows where 'text' is missing (NaN)
missing_text_rows = df_posts[df_posts['text'].isnull()]
missing_text_rows

,timestamp,text,text_id,user,user_id,hashtags,mentions,emojis,frequency,language


### Write the output to csv file

In [23]:
df_posts.to_csv('../output/preprocessed.csv', index=False)

In [24]:
df_posts[df_posts['frequency'] > 85]

,timestamp,text,text_id,user,user_id,hashtags,mentions,emojis,frequency,language
48,00:00:19,wwf mercy usa variant nintendo n cart mint,2053226516,reginabarnes,1068051022,[],[],[],86,hu
10602,02:57:00,climate change effective way stop tsunami torn...,2016959704,smithcynthia,1055454345,[],[],[],116,en
45453,12:32:46,difference degree global warming,2011488077,raymondberry,1031499948,"[#ClimateCrisis, #ClimateEmergency, #ClimateAc...",[],[],119,en
